In [125]:
import time
import email
import smtplib
import imaplib
import mailparser
import mysql
import mysql.connector


conn=mysql.connector.connect(host="localhost",user="root",password="abcd@1234")
crs=conn.cursor()
crs.execute('use mpr')
insert='insert into db values(null,%s,%s,%s,%s,%s,%s,%s,%s)'

def read_email_from_gmail(crs):
    try:
        server = imaplib.IMAP4_SSL('imap.gmail.com')
        server.login("garglakshay631@gmail.com","abcd@1234")
        server.select('inbox')

        typ, data = server.search(None, 'ALL')
        print("Connection",typ)
        print(data)
        mail_ids = data[0]
        
        id_list = mail_ids.split()  
        first_email = int(id_list[0])
        latest_email = int(id_list[-1])
        for i in range(latest_email,first_email-1,-1):
            typ, data = server.fetch(str(i), '(RFC822)' )
            for response_part in data:
                if isinstance(response_part, tuple):
                    mail = mailparser.parse_from_bytes(response_part[-1])
                    name=mail.subject
                    to=mail.to[-1][-1]
                    from_=mail.from_[-1][-1]
                    att=mail.attachments
                    body=mail.text_plain[-1]
                    b=body.split('\r\n')
                    contact=b[-1]
                    date=b[-3]
                    venue=b[-5]
                    reg=b[-7]
                    str1=" ".join(b[:-7])
                    print(from_,to,name,str1,date,venue,reg,contact)
#                     crs.execute(insert,val)
#         conn.commit()

    except (Exception, e):
        print (str(e))

In [126]:
read_email_from_gmail(crs)

Connection OK
[b'1']
lakshay04014902016@msi-ggsip.org garglakshay631@gmail.com Chessmate Chess is life in a miniature. Chess is struggle, Chess is battles.                        -Garry Kasparov  Get yourselves prepared whilst we, the students of MAHARAJA SURAJMAL INSTITUTE present you with CHESSMATE ,  the ultimate CHESS TOURNAMENT which will knock you out from all sides.  An event to provide you with the opportunity to showcase your wit, intellect, strategic ability and an exercise to your mind  -all in an exciting but intense experience.  Because it’s all fun and games until it comes to checking your mates out.   Participate and paint a black and white picture of who's The King , The Knight in shining armour and the ones who turn out to be just Pawns.   SO PLAN YOUR STRATEGY AND MAKE YOUR MOVE!  22 September 2018 Room No- 110,104, 1st Floor, MSI https://goo.gl/forms/OwQQEENQ1eQ8QNQA3 Ansh Bhatia: 8810516070


In [63]:
conn=mysql.connector.connect(host="localhost",user="root",password="abcd@1234")
crs=conn.cursor()        

In [35]:
crs.execute('show databases')
for x in crs:
    print(x)

('information_schema',)
('MPR',)
('mysql',)
('performance_schema',)
('sys',)


In [36]:
crs.execute('use mpr')
crs.execute('show tables')
for x in crs:
    print(x)

In [27]:
crs.execute('drop table db')
crs.execute('describe db')
for x in crs:
    print(x)

ProgrammingError: 1146 (42S02): Table 'mpr.db' doesn't exist